# **DL4H598 Final Project**
# Project Team #44 - Miles Kopcke and Sergio Rio
# {mkopcke2, sav7}@illinois.edu
# Paper ID #12 - ["Using recurrent neural network models for early detection of heart failure onset"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5391725/)
# Edward Choi, Andy Schuetz, Walter F Stewart, and Jimeng Sun

## Overview
## The scope of this project is to reproduce the main  results claimed by the authors to be “state-of-the-art prediction performance,” outperforming traditional machine learning models. We will run experiments with the two GRU models (with and without time duration information) and the MLP model described in the research paper for all three types of inputs (one-hot encoding, grouped codes, and medical concept vectors based on Skip-gram). 
## The original study used data from Sutter Palo Alto Medical Foundation (Sutter-PAMF) primary care patients from May 16, 2000, to May 23, 2023. This data was used to construct 3,884 Heart Failure cases and 28,903 controls. Currently, we’re pursuing the use of the IBM MarketScan Research Database (Merative n.d.) as one of our team members works for IBM Watson Health (now called Merative). This would allow us to have a very similar patient population to the original study.

####  *Choi, Edward, Andy Schuetz, Walter F Stuart, and Jimeng Sun. 2016. "Using recurrent neural network models for early detection of heart failure onset." Journal of the American Medical Informatics Association 24 (2): 361-369. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5391725/.


## **1. Data preparation**
## Develop, test, and execute programs for dataset sampling of cases and controls, engineer data into the three types of encoding (one-hot encoding, grouped codes, and medical concept vectors based on Skip-gram), and iteratively divide the data into train, validation, and test with a ratio of 5:1:1.


## We’re using the IBM MarketScan Research Database (Merative n.d.). 
## This allows us to have a very similar patient population to the original study. Specifically, we’d be using the years 2011 to 2013, composed of 66,020,609 patients in the outpatient setting with a total of 3,120,102,668 encounters, giving us an average number of 47 encounters per patient. We adopted the same density sampling design used for the longitudinal records of the research paper  (Choi, et al. 2016), section Definitions of cases and controls to ensure we obtain a similar data sample.
## MarketScan data was extracted from a Snowflake datalake, exported to csv files and processed in Python using Pandas library. 
## Plese check Market_Scan_Data_Extraction.sql file in https://github.com/svianrio/HF-Prediction/


## Using small dataset to train and test models

We will use a dataset synthesized from [MIMIC-III](https://mimic.physionet.org/gettingstarted/access/).

In [ ]:
import os
import pdb
import sys
import pickle
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split


# set random seed for reproducibility
# “the Ultimate Question of Life, the Universe, and Everything”
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.use_deterministic_algorithms(True)
os.environ['PYTHONASHSEED'] = str(seed)

# define data set path
DATA_PATH = '/Users/mkopcke/Documents/cs598'

# define number of timesteps to be used for model development
global timesteps
timesteps = 100

In [ ]:
hfs = pd.read_csv('input_hfs.csv')["Y"].tolist()

seqs = pickle.load(open(os.path.join(DATA_PATH,'input_seqs.pkl'), 'rb'))

print(len(hfs))
print(len(seqs))
seqs[0]

54154
54154


[[19, 8693, 46],
 [467, 8693],
 [9900, 492, 241, 822, 9015],
 [202, 83, 15398, 8695],
 [83, 621, 846, 8695],
 [9729, 19, 15350, 15551],
 [15350, 15551],
 [865, 83, 19, 9395, 8695],
 [15350, 15551]]

In [ ]:
# define maxium number of codes
global MAX_CODES
# MAX_CODES = len(types)
MAX_CODES = 18945

## Dataloader Class

We will use the sequences of diagnosis codes `seqs` as input and heart failure `hfs` as output.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, seqs, hfs):

        self.x = seqs
        self.y = hfs
    
    def __len__(self):

        return len(self.y)

    def __getitem__(self, index):

        return self.x[index], self.y[index]


In [ ]:
dataset = CustomDataset(seqs, hfs)

In [ ]:
def load_data(train_dataset, val_dataset, batch_size,collate_fn):
    
    train_loader = DataLoader(train_dataset, batch_size, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size, collate_fn=collate_fn)
    
    return train_loader, val_loader

##  Data Collator for multi-hot vectors

For example, when the `DataLoader` gets a list of two samples.

```
[ [ [0, 1, 2], [3, 0] ], 
  [ [1, 3, 6, 3], [2], [3, 1] ] ]
```

where the first sample has two visits `[0, 1, 2]` and `[3, 0]` and the second sample has three visits `[1, 3, 6, 3]`, `[2]`, and `[3, 1]`.

The collate function `Collator()` is supposed to concatenate all visits of one patient together to form the inputs, as

```
[[0, 1, 2, 3 ,0],
 [1, 3, 6, 3, 2, 3 ,1]]

```

Further, we transform this to a multi-hot vector representing the appearances of events. Suppose we the number of all possible events is 6, the yielded outputs should be

```
[[0, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 0, 1]]
```
which will be the final inputs.



In [ ]:
class MultiHotCollator:
    def __init__(self, total_number_of_codes):
        self.max_num_codes = total_number_of_codes
        
    def __call__(self, data):
        
        sequences, labels = zip(*data)
        num_patients = len(sequences)
        num_visits = [len(patient) for patient in sequences]
        num_codes = [len(visit) for patient in sequences for visit in patient]

        max_num_visits = max(num_visits)
        max_num_codes = self.max_num_codes
        
        y = torch.tensor(labels, dtype=torch.float)
        x = torch.zeros((num_patients, timesteps, max_num_codes), dtype=torch.float)
        masks = torch.zeros((num_patients, timesteps,1), dtype=torch.float)
        
        for i_patient, patient in enumerate(sequences):
            y[i_patient] = labels[i_patient]
            
            for visit_number, visit in enumerate(patient):
              
              if visit_number < timesteps: 
                idx = torch.tensor([visit], dtype=torch.int64)
                multihot = torch.zeros((1,max_num_codes), dtype=torch.float)
                x[i_patient, visit_number,:] = multihot.scatter_(dim=1, index=idx, src=torch.ones((idx.size(0),idx.size(1)),dtype=torch.float))
                masks[i_patient, visit_number] = 1
            
                
        return x, masks, y



In [ ]:
collate_fn = MultiHotCollator(MAX_CODES)

In [ ]:
dataset = CustomDataset(seqs, hfs)
a,b=dataset[0]

In [ ]:
dataset = CustomDataset(seqs, hfs)

In [ ]:
split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 800
Length of val dataset: 200


In [ ]:
train_loader, val_loader = load_data(train_dataset, val_dataset, batch_size=10, collate_fn=collate_fn)

In [ ]:
batch_size = 32
train_loader, val_loader = load_data(train_dataset, val_dataset, batch_size, collate_fn)

In [ ]:
dataset = CustomDataset(seqs, hfs)
collate_fn = MultiHotCollator(MAX_CODES)
loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn, shuffle=False)
loader_iter = iter(loader)
c, m, d = next(loader_iter)

##  Data Collator for Embeddings - Padding and Masking

In [ ]:
def PadMaskCollator(data):

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        last_visit = 0
        for j_visit, visit in enumerate(patient):
            a_codes = 0
            for k_code, code in enumerate(visit):
                x[i_patient,j_visit,k_code] = code
                masks[i_patient,j_visit,k_code] = True
                a_codes = k_code
                last_visit = j_visit
    
    return x, masks, y

In [ ]:
dataset = CustomDataset(seqs, hfs)
loader = DataLoader(dataset, batch_size=32, collate_fn=PadMaskCollator)
loader_iter = iter(loader)
x, masks, y = next(loader_iter)

In [ ]:
def sum_embeddings_with_mask(x, masks):

    samples = x.size(0)
    visits = x.size(1)
    diag_codes = x.size(2)
    emb_dim = x.size(3)
    sum_embeddings = torch.zeros((samples,visits, emb_dim), dtype=torch.long)
    masks = torch.unsqueeze(masks,3)
    x = x * masks.expand([samples, visits, diag_codes, emb_dim])

    return torch.sum(x,2,keepdim=False)

In [ ]:
def get_last_visit(hidden_states, masks):

    batch_size = hidden_states.size(0)
    len_true_visit = masks.sum(dim = 2)
    nz = len_true_visit.count_nonzero(dim=1)

    return hidden_states[range(0,batch_size),nz-1,:]

##  Data Collator Aggregator - Required for MLP Model

In [ ]:
def Aggregator(data):

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, MAX_CODES), dtype=torch.float)

    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            for k_code, code in enumerate(visit):
                x[i_patient, code] += 1
    
    m = torch.mean(x)
    x = (x - m) / x.max()
    
    return x, y

In [ ]:
dataset = CustomDataset(seqs, hfs)
loader = DataLoader(dataset, batch_size=32, collate_fn=Aggregator)
loader_iter = iter(loader)
x, y = next(loader_iter)

In [ ]:
x.size()

torch.Size([32, 18945])

## **2. Model development**
## Iteratively develop and test models, starting with MLP, GRU without duration information, and finally, GRU with duration information for all input types.

###  **Models with their corresponding hyper-parameters**
### **MLP with one-hot vectors -** L2 regularization: 0.01, Hidden layer size: 15, Max epoch: 100
### **MLP with grouped code vectors & medical concept vectors -** L2 regularization: 0.001, Hidden layer size: 100, Max epoch: 100
### **All GRU models -** L2 regularization: 0.001, Hidden layer size: 100, Max epoch: 100

In [ ]:
class MLP(nn.Module):
  def __init__(self, num_units):
        """
        MLP 
        layer1 with num_codes units
        layer2 with 15 units
             
        """
        
        super().__init__()
        self.fc1 = nn.Linear(num_units,15)
        self.fc2 = nn.Linear(15,1)
        self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):

    batch_size = x.shape[0]
    x = self.fc1(x)
    logits = self.fc2(x)        
    probs = self.sigmoid(logits)
    return probs.view(batch_size)

In [ ]:
class GRU(nn.Module):
  def __init__(self, num_codes,dp_out=0):
        """
        RNN with 1 layer and 100 GRU units
        """
        
        super().__init__()
        # self.embedding = nn.Embedding(num_codes,timesteps)
        self.gru = nn.GRU(input_size = MAX_CODES, hidden_size= timesteps, num_layers = 1, bias=True, batch_first=True)
        if dp_out > 0:
          self.dropout = nn.Dropout(dp_out)
          self.dpout = True
        else:
          self.dpout = False
        self.fc = nn.Linear(timesteps,1,bias=False)
        self.sigmoid = nn.Sigmoid()
  
  def forward(self, x, masks):

    batch_size = x.shape[0]
    # x = self.embedding(x)
    # x = sum_embeddings_with_mask(x, masks)

    # We only care about the last status of the hidden layer
    output, h_n = self.gru(x)
    true_h_n = get_last_visit(output, masks)
    if self.dpout:
      true_h_n = self.dropout(true_h_n)
    logits = self.fc(true_h_n)        
    probs = self.sigmoid(logits)

    return probs.view(batch_size)


## **3. Performance Evaluation**
## Evaluate AUC scores of developed models and measure inference times for a single patient. 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval_model_with_mask(model, val_loader):
    
    """
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, y in val_loader:
        y_hat = model(x, masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)

    p, r, f, s = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score, average='samples')

    return p, r, f, roc_auc

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval_model_without_mask(model, val_loader):
    
    """
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, y in val_loader:
        y_hat = model(x)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)

    p, r, f, s = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score, average='samples')

    return p, r, f, roc_auc

In [ ]:
def train_with_mask(model, train_loader, val_loader, n_epochs):
    """
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks,y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x,masks)
            loss = criterion(torch.squeeze(y_hat),y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval_model_with_mask(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
              .format(epoch+1, p, r, f, roc_auc))

In [ ]:
def train_without_mask(model, train_loader, val_loader, n_epochs):
    """
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x,y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(torch.squeeze(y_hat),y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval_model_without_mask(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
              .format(epoch+1, p, r, f, roc_auc))

## **3.1 MLP Model Training**

In [ ]:
dataset = CustomDataset(seqs, hfs)
split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 43323
Length of val dataset: 10831


In [ ]:
train_loader, val_loader = load_data(train_dataset, val_dataset, batch_size=32, collate_fn=Aggregator)

In [ ]:
mlp = MLP(MAX_CODES)

In [ ]:
criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(mlp.parameters(),lr=0.001,weight_decay=1e-5)

In [ ]:
n_epochs = 10
train_without_mask(mlp, train_loader, val_loader, n_epochs)

Epoch: 1 	 Training Loss: 0.325722
Epoch: 1 	 Validation p: 0.75, r:0.24, f: 0.36, roc_auc: 0.92
Epoch: 2 	 Training Loss: 0.262024
Epoch: 2 	 Validation p: 0.79, r:0.36, f: 0.49, roc_auc: 0.93
Epoch: 3 	 Training Loss: 0.242696
Epoch: 3 	 Validation p: 0.82, r:0.44, f: 0.58, roc_auc: 0.93
Epoch: 4 	 Training Loss: 0.229201
Epoch: 4 	 Validation p: 0.84, r:0.51, f: 0.64, roc_auc: 0.94
Epoch: 5 	 Training Loss: 0.217646
Epoch: 5 	 Validation p: 0.85, r:0.57, f: 0.68, roc_auc: 0.94
Epoch: 6 	 Training Loss: 0.213966
Epoch: 6 	 Validation p: 0.86, r:0.62, f: 0.72, roc_auc: 0.94
Epoch: 7 	 Training Loss: 0.206621
Epoch: 7 	 Validation p: 0.86, r:0.65, f: 0.74, roc_auc: 0.94
Epoch: 8 	 Training Loss: 0.209387
Epoch: 8 	 Validation p: 0.88, r:0.67, f: 0.76, roc_auc: 0.95
Epoch: 9 	 Training Loss: 0.215973
Epoch: 9 	 Validation p: 0.88, r:0.69, f: 0.77, roc_auc: 0.95
Epoch: 10 	 Training Loss: 0.216221
Epoch: 10 	 Validation p: 0.88, r:0.71, f: 0.79, roc_auc: 0.95


## **3.2 GRU Model Training**


In [ ]:
dataset = CustomDataset(seqs, hfs)
split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 43323
Length of val dataset: 10831


In [ ]:
train_loader, val_loader = load_data(train_dataset, val_dataset, batch_size=32, collate_fn=MultiHotCollator(MAX_CODES))

In [ ]:
gru = GRU(MAX_CODES, dp_out=0.1)

In [ ]:
criterion =  nn.BCELoss()
optimizer = torch.optim.Adam(gru.parameters(),lr=0.001,weight_decay=1e-5)

In [ ]:
# number of epochs to train the model
n_epochs = 10
train_with_mask(gru, train_loader, val_loader, n_epochs)

Epoch: 1 	 Training Loss: 0.112160
Epoch: 1 	 Validation p: 0.99, r:0.89, f: 0.93, roc_auc: 0.99
Epoch: 2 	 Training Loss: 0.038666
Epoch: 2 	 Validation p: 0.99, r:0.90, f: 0.94, roc_auc: 0.99
Epoch: 3 	 Training Loss: 0.020708
Epoch: 3 	 Validation p: 0.98, r:0.91, f: 0.94, roc_auc: 0.99
Epoch: 4 	 Training Loss: 0.011672
Epoch: 4 	 Validation p: 0.99, r:0.89, f: 0.94, roc_auc: 0.99
Epoch: 5 	 Training Loss: 0.006672
Epoch: 5 	 Validation p: 0.97, r:0.91, f: 0.94, roc_auc: 0.99
Epoch: 6 	 Training Loss: 0.006484
Epoch: 6 	 Validation p: 0.99, r:0.91, f: 0.94, roc_auc: 0.98
Epoch: 7 	 Training Loss: 0.005432
Epoch: 7 	 Validation p: 0.98, r:0.91, f: 0.94, roc_auc: 0.98
Epoch: 8 	 Training Loss: 0.004702
Epoch: 8 	 Validation p: 0.96, r:0.91, f: 0.94, roc_auc: 0.99
Epoch: 9 	 Training Loss: 0.003883
Epoch: 9 	 Validation p: 0.96, r:0.91, f: 0.94, roc_auc: 0.99
Epoch: 10 	 Training Loss: 0.003327
Epoch: 10 	 Validation p: 0.96, r:0.91, f: 0.93, roc_auc: 0.99
